In [7]:
import lda
from scipy import sparse
import numpy as np
import json
import gensim 

In [2]:
X_train = sparse.load_npz('/Users/tingchen/Desktop/Yelp Project/data/X_train.npz')
cc= np.load("/Users/tingchen/Desktop/Yelp Project/data/y_train.npy")

In [3]:
ix = X_train.getnnz(1)>0
X_train = X_train[ix]
cc= cc[ix]

In [8]:
model = lda.LDA(n_topics=10, n_iter=1500)
model.fit(X_train.astype(np.int32), cc.astype(np.int32))

all zero column in document-term matrix found
/Users/tingchen/Desktop/Yelp_Cultural_Analysis/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):


Finished initializing
0/1500
10/1500
20/1500
30/1500
40/1500
50/1500
60/1500
70/1500
80/1500
90/1500
100/1500
110/1500
120/1500
130/1500
140/1500
150/1500
160/1500
170/1500
180/1500
190/1500
200/1500
210/1500
220/1500
230/1500
240/1500
250/1500
260/1500
270/1500
280/1500
290/1500
300/1500
310/1500
320/1500
330/1500
340/1500
350/1500
360/1500
370/1500
380/1500
390/1500
400/1500
410/1500
420/1500
430/1500
440/1500
450/1500
460/1500
470/1500
480/1500
490/1500
500/1500
510/1500
520/1500
530/1500
540/1500
550/1500
560/1500
570/1500
580/1500
590/1500
600/1500
610/1500
620/1500
630/1500
640/1500
650/1500
660/1500
670/1500
680/1500
690/1500
700/1500
710/1500
720/1500
730/1500
740/1500
750/1500
760/1500
770/1500
780/1500
790/1500
800/1500
810/1500
820/1500
830/1500
840/1500
850/1500
860/1500
870/1500
880/1500
890/1500
900/1500
910/1500
920/1500
930/1500
940/1500
950/1500
960/1500
970/1500
980/1500
990/1500
1000/1500
1010/1500
1020/1500
1030/1500
1040/1500
1050/1500
1060/1500
1070/1500
1080/1500

In [5]:
dic = gensim.corpora.Dictionary.load("/Users/tingchen/Desktop/Yelp Project/big_lexicon")

1500 iterations

In [9]:
n_top_words = 15
topic_word = model.topic_word_  
collection_word = model.topic_word_collection_ 
collections = ['Chinese', 'Italian', 'Mexican']

lexicon = np.array(list(dic.values()))

for i, topic_dist in enumerate(topic_word):
    topic_words = lexicon[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}\n'.format(i, ' '.join(topic_words)))
    
    collection_words = collection_word[i, :, :]
    
    for i, eth in enumerate(collections):
        topic_words = lexicon[np.argsort(collection_words[:, i])][:-n_top_words:-1]
        print('Cuisine {}: {}'.format(eth, ' '.join(topic_words)))
    print("\n")

Topic 0: i the good n't 's food it like place one also really cheese ordered

Cuisine Chinese: tea buffet milk sushi ice dessert lobster drink rice crab seafood cream style drinks
Cuisine Italian: italian pasta fresh gelato pizza cream dessert mozzarella tomato italy chocolate prosciutto oil wine
Cuisine Mexican: salsa burrito chips beans chicken mexican green tortilla guacamole fresh cream red enchilada taco


Topic 1: i 's the n't food place good like one it restaurant get time go

Cuisine Chinese: chinese pho vegas panda location asian club night best always express thai vietnamese authentic
Cuisine Italian: vegas wine room italian music hotel old strip drinks patio happy fun hour las
Cuisine Mexican: mexican place years always vegas love chips family taco business night going open el


Topic 2: i the good n't food 's it like place we would ordered really one

Cuisine Chinese: dim sum duck pork congee shrimp rice bbq fried chinese buns carts dumplings steamed
Cuisine Italian: pasta 

1000 iterations

In [6]:
n_top_words = 8
topic_word = model.topic_word_  
collection_word = model.topic_word_collection_ 
collections = ['Chinese', 'Italian', 'Mexican']

lexicon = np.array(list(dic.values()))

for i, topic_dist in enumerate(topic_word):
    topic_words = lexicon[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}\n'.format(i, ' '.join(topic_words)))
    
    collection_words = collection_word[i, :, :]
    
    for i, eth in enumerate(collections):
        topic_words = lexicon[np.argsort(collection_words[:, i])][:-n_top_words:-1]
        print('Cuisine {}: {}'.format(eth, ' '.join(topic_words)))
    print("\n")

Topic 0: i the n't 's good food like

Cuisine Chinese: pho thai buffet chinese sushi rolls pad
Cuisine Italian: delicious italian wine pizza great fresh dessert
Cuisine Mexican: burrito salsa beans chicken rice chips cheese


Topic 1: i the 's n't place good food

Cuisine Chinese: tea milk 's drink get drinks place
Cuisine Italian: place room 's bar vegas hotel drinks
Cuisine Mexican: mexican food '' bar restaurant salsa people


Topic 2: i the n't good 's food place

Cuisine Chinese: dim sum duck pork shrimp chinese dishes
Cuisine Italian: great food we service the amazing delicious
Cuisine Mexican: we great chips service us margarita server


Topic 3: i the n't 's good like place

Cuisine Chinese: noodles soup noodle beef dumplings pork spicy
Cuisine Italian: pizza crust cheese good sauce the i
Cuisine Mexican: tacos taco asada carne fish meat pastor


Topic 4: i the n't 's good like it

Cuisine Chinese: chicken rice fried chinese sauce beef egg
Cuisine Italian: pasta sauce the bread